# Final Project: Huggingface Transformers를 이용한 한-영 번역

## 개요
- Assignment2의 연장선에서 한-영 번역 프로그램을 Huggingface의 Transformers와 Dataset을 이용해서 필요한 크기로 데이터를 줄이고 이를 모델에서 파인튜닝하여 번역 성능을 살펴보고 실제 예문의 번역을 출력
- [Huggingface NLP course의 7장 Translation](https://huggingface.co/learn/nlp-course/chapter7/4?fw=pt)을 근간으로 해서 (거의 그대로 활용할 수 있음) 구현할 수 있음
- Dataset을 자료를 받아서 필요한 크기로 나누고, 학습에 필요한 형태로 Dataset을 재구조화하고 tokenize하는 모듈을 구현
- 공개된 자료를 바탕으로 구현하기 때문에 성능보다는 전체 번역모듈을 Huggingface로 구현해보는 것을 주목표로 하기 때문에 완결성이 있어야 하며, 실제로 작동해야 함.
- FinalProject_학번_이름.ipynb
- Due 12월 8일 11시 59분

## 필요한 모듈 설치
- 프로그램 실행에 필요한 모듈, Huggingface, Dataset 등을 각자 알아서 설치

## Dataset
- Huggingface Hub에 있는 Dataset 중 `bongsoo/news_talk_en_ko` 는 한국어-영어뉴스 기사를 병렬로 작성한 130만 개의 데이터 셋이다.
- 이 데이터셋을 읽어서 colab에서 돌릴 수 있게, training, validation, test 데이터로 각각 120,000, 9,000, 1,000으로 줄여서 학습에 필요한 구조로 만듬
- 데이터를 자를때 순차적으로 자르지 말고 전체 데이터를 셔플한 후 필요한 크기로 자를 것
- 데이터셋을 pandas 형식으로 받은 후 할 수도 있고 여러 가능한 방법이 있음

In [ ]:
# 필요한 라이브러리 설치 및 로드
from datasets import load_dataset, Dataset
import pandas as pd

# 데이터셋 로드 및 처리
dataset = load_dataset("bongsoo/news_talk_en_ko")
df = pd.DataFrame(dataset['train']).sample(frac=1, random_state=42).reset_index(drop=True)

# 데이터 크기 설정
train_df = df.iloc[:120_000]
val_df = df.iloc[120_000:129_000]
test_df = df.iloc[129_000:130_000]

# # 데이터 저장
# train_df.to_csv("train.csv", index=False)
# val_df.to_csv("val.csv", index=False)
# test_df.to_csv("test.csv", index=False)

train_dataset = Dataset.from_pandas(train_df)
valid_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

Generating train split:   0%|          | 0/1299999 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ["Skinner's reward is mostly eye-watering.", '스키너가 말한 보상은 대부분 눈으로 볼 수 있는 현물이다.'],
        num_rows: 1299999
    })
})

## Huggingface
- 학습에 필요한 Huggingface 모델 사용
- AutoTokenizer, AutoModelForSeq2SeqLM 등을 사용
- 학습에 사용할 모델은 [T5](https://github.com/AIRC-KETI/ke-t5)("KETI-AIR/ke-t5-base")를 사용할 것

In [ ]:
# Huggingface 모델 및 토크나이저 로드
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# 모델 이름 설정
model_name = "KETI-AIR/ke-t5-base"

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 모델과 토크나이저 준비 완료 확인
print(f"Model {model_name} and tokenizer loaded successfully!")

## Tokenizer
- T5는 sentencepiece tokenizer를 사용하기 때문에 한-영 병렬 데이터의 자료를 학습시키기 위해서는 이 데이터를 tokenizer를 써서 프로세싱을 해야 한다. 이를 위한 모듈을 만들고 한국어, 영어데이터를 tokenize하여 모델에 입력할 수 있는 형태로(tokenized-dataset) 바꾼다
- 이를 위해서 Dataset의 map()을 활용하도록 한다.

In [ ]:
def preprocess_function(examples):
    # 토큰화: source와 target 각각에 대해 처리
    inputs = examples["ko"]  # 한국어 데이터
    targets = examples["en"]  # 영어 데이터
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    # 레이블 토큰화
    labels = tokenizer(targets, max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

train_dataset = train_dataset.map(preprocess_function, batched=True, num_proc = 2)
valid_dataset = valid_dataset.map(preprocess_function, batched=True, num_proc = 2)
test_dataset = test_dataset.map(preprocess_function, batched=True, num_proc = 2)


## Model
- 학습에 필요한 모델 설정

## Collator
- 학습할 자료를 정렬하고 모델에 배치 단위로 넘겨주기 위해 준비

## Metric
- 학습한 모델을 측정할 매트릭을 준비
- 번역 모델에서는 주로 BLEU 점수를 사용
- BLEU 점수는 번역기가 생성한 문장이 레퍼런스(정답이라는 표현을 사용하지 않는 이유는 제대로 된 번역 문장이 오직 하나가 아니기 때문)문장과 얼마나 비슷한지 측정하는 점수

- sacrebleu 라이브러리는 BLEU 구현체에서 사실상 표준 라이브러리이며 각 모델이 다른 토크나이저를 쓰는 경우 이를 BPE로 통일 시켜 BLEU 점수를 계산



## 모델 학습(Train)
- 학습을 간단히 하기위해 허깅페이스에서 제공하는 Seq2SeqTrainer클래스와 학습 세부 조건은 Seq2SeqTrainingArguments를 활용할 수 있으나, 본 과제에서는 이를 쓰지 말고 Training를 직접 구현하도록 한다. Dataloader, Scheduler, ACCELERATOR, Optimizer 등을 설정하고 실제로 training loop를 돌려서 학습하고, evaluation 데이터로 성능을 검증
- colab에서 돌리기 위해서는 성능이 저하되겠지만, batch size 등을 적당하게 설정해야 함.

In [ ]:
import torch
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, AdamW, get_scheduler, DataCollatorForSeq2Seq
from datasets import Dataset

# 모델 및 데이터 로드
model_name = "KETI-AIR/ke-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# DataLoader with collator
train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=data_collator)
val_dataloader = DataLoader(valid_dataset, batch_size=8, collate_fn=data_collator)


# Optimizer 및 Scheduler 설정
optimizer = AdamW(model.parameters(), lr=5e-5)
num_training_steps = len(train_dataloader) * 3  # 3 에폭 예시
lr_scheduler = get_scheduler("linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

# Accelerator 설정 (GPU 사용)
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)

# Training Loop
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    train_loss = 0
    for batch in train_dataloader:
        # 배치를 GPU로 이동
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        # Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        train_loss += loss.item()

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()

    avg_train_loss = train_loss / len(train_dataloader)
    print(f"Training loss: {avg_train_loss}")

    # Evaluation Loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for batch in val_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

    avg_val_loss = val_loss / len(val_dataloader)
    print(f"Validation loss: {avg_val_loss}")

print("Training completed!")



In [ ]:
from evaluate import load

# Load BLEU metric
bleu = load("bleu")

# Function to calculate BLEU score
def compute_bleu_with_evaluate(predictions, references):
    """
    predictions: List of model-generated translations
    references: List of reference translations
    """
    results = bleu.compute(predictions=predictions, references=[[ref] for ref in references])
    return results["bleu"]

# Evaluation Loop for BLEU Calculation
model.eval()
predictions = []
references = []

with torch.no_grad():
    for batch in val_dataloader:
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"]

        # Generate predictions
        outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
        decoded_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
        decoded_references = tokenizer.batch_decode(labels, skip_special_tokens=True)

        # Collect predictions and references
        predictions.extend(decoded_predictions)
        references.extend(decoded_references)

# Calculate BLEU score
bleu_score = compute_bleu_with_evaluate(predictions, references)
print(f"BLEU score: {bleu_score}")


## 모델 테스트 (Test)
- 학습된 모델을 가지고 테스트 데이터로 테스트


In [ ]:
# Function to evaluate model on test data
def evaluate_model_on_test_data(model, tokenizer, test_dataloader):
    model.eval()
    predictions = []
    references = []

    with torch.no_grad():
        for batch in test_dataloader:
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"]

            # Generate predictions
            outputs = model.generate(input_ids=input_ids, attention_mask=attention_mask, max_length=50)
            decoded_predictions = tokenizer.batch_decode(outputs, skip_special_tokens=True)
            decoded_references = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # Collect predictions and references
            predictions.extend(decoded_predictions)
            references.extend(decoded_references)

    # Calculate BLEU score
    bleu_score = compute_bleu_with_evaluate(predictions, references)
    return bleu_score, predictions, references


# Prepare test DataLoader
from torch.utils.data import DataLoader
test_dataloader = DataLoader(test_dataset, batch_size=8, collate_fn=data_collator)

# Evaluate on test data
test_bleu_score, test_predictions, test_references = evaluate_model_on_test_data(model, tokenizer, test_dataloader)

# Print the BLEU score
print(f"Test BLEU score: {test_bleu_score}")



## Inference

- Assignment2에 쓰였던 문장들을 이 학습된 모델에서 그 결과를 살펴 보아라

- 모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.
- 미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?
- 은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔시세요
- 아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?
- 부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다
- 변기가 막혔습니다
- 그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?
- 비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.
- 속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다
- 문대통령은 집단 이익에서 벗어나라고 말했다
- 이것 좀 먹어 볼 몇 일 간의 시간을 주세요
- 이 날 개미군단은 외인의 물량을 모두 받아 내었다
- 통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다
- 이번 구조 조정이 제품을 효과적으로 개발 하고 판매 하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다
- 요즘 이 프로그램 녹화하며 많은 걸 느낀다

In [ ]:
# Test sentences to translate
test_sentences = [
    "모든 액체, 젤, 에어로졸 등은 1커트짜리 여닫이 투명봉지 하나에 넣어야 합니다.",
    "미안하지만, 뒷쪽 아이들의 떠드는 소리가 커서, 광화문으로 가고 싶은데 표를 바꾸어 주시겠어요?",
    "은행이 너무 멀어서 안되겠네요. 현찰이 필요하면 돈을 훔치세요.",
    "아무래도 분실한 것 같으니 분실 신고서를 작성해야 하겠습니다. 사무실로 같이 가실까요?",
    "부산에서 코로나 확진자가 급증해서 병상이 부족해지자 확진자 20명을 대구로 이송한다.",
    "변기가 막혔습니다.",
    "그 바지 좀 보여주십시오. 이거 얼마에 살 수 있는 것 입니까?",
    "비가 와서 백화점으로 가지 말고 두타로 갔으면 좋겠습니다.",
    "속이 안좋을 때는 죽이나 미음으로 아침을 대신합니다.",
    "문대통령은 집단 이익에서 벗어나라고 말했다.",
    "이것 좀 먹어 볼 몇 일 간의 시간을 주세요.",
    "이 날 개미군단은 외인의 물량을 모두 받아 내었다.",
    "통합 우승의 목표를 달성한 NC 다이노스 나성범이 메이저리그 진출이라는 또 다른 꿈을 향해 나아간다.",
    "이번 구조 조정이 제품을 효과적으로 개발하고 판매하기 위한 회사의 능력 강화 조처임을 이해해 주시리라 생각합니다.",
    "요즘 이 프로그램 녹화하며 많은 걸 느낀다."
]

# Tokenize test sentences
inputs = tokenizer(test_sentences, return_tensors="pt", padding=True, truncation=True, max_length=512)

# Move inputs to the same device as the model
inputs = {key: value.to(device) for key, value in inputs.items()}

# Generate translations
model.eval()
with torch.no_grad():
    translated_ids = model.generate(**inputs, max_length=100, num_beams=4)
    translations = tokenizer.batch_decode(translated_ids, skip_special_tokens=True)

# Print translations
for i, (src, tgt) in enumerate(zip(test_sentences, translations)):
    print(f"Source {i+1}: {src}")
    print(f"Translation {i+1}: {tgt}\n")
